# AutoCron Demo 2: Advanced Features

This notebook demonstrates advanced scheduling features.

## Features Covered:
- Retries and error handling
- Timeouts
- Task priorities
- Task dependencies
- Task metadata and tags

## Setup

In [ ]:
from autocron import AutoCron, schedule
from datetime import datetime
import time
import random

## 1. Retries and Error Handling

Tasks can automatically retry on failure.

In [ ]:
# Task that might fail
attempt_count = 0

@schedule(every='10s', retries=3, retry_delay=5)
def unreliable_api_call():
    """Simulates an unreliable API that might fail."""
    global attempt_count
    attempt_count += 1
    
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Attempt #{attempt_count}")
    
    # Simulate random failures
    if random.random() < 0.6:  # 60% failure rate
        print("  ❌ API call failed!")
        raise Exception("Connection timeout")
    else:
        print("  ✅ API call succeeded!")
        return "Data retrieved"

print("✅ Task with retries scheduled!")
print("📝 Will retry up to 3 times with 5 second delay between attempts")

In [ ]:
# Using scheduler class for more control
scheduler = AutoCron()

def flaky_task():
    if random.random() < 0.5:
        raise Exception("Random failure")
    return "Success"

scheduler.add_task(
    name="flaky_task",
    func=flaky_task,
    every='15s',
    retries=5,           # Retry up to 5 times
    retry_delay=3        # Wait 3 seconds between retries
)

print("✅ Task added with 5 retries and 3 second delay")

## 2. Timeouts

Prevent tasks from running too long.

In [ ]:
@schedule(every='20s', timeout=10)
def long_running_task():
    """Task that might take too long."""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Starting long task...")
    
    # Simulate long processing
    duration = random.randint(5, 15)
    print(f"  Processing for {duration} seconds...")
    time.sleep(duration)
    
    print("  ✅ Task completed!")
    return "Done"

print("✅ Task with 10-second timeout scheduled!")
print("⏱️  If task takes longer than 10 seconds, it will be terminated")

In [ ]:
# Combining timeout with retries
scheduler = AutoCron()

def slow_processing():
    time.sleep(random.randint(3, 8))
    return "Processed"

scheduler.add_task(
    name="slow_processor",
    func=slow_processing,
    every='30s',
    timeout=5,           # 5 second timeout
    retries=2,           # Retry if timeout
    retry_delay=2
)

print("✅ Task with timeout + retries configured")

## 3. Task Priorities

Control execution order when multiple tasks are ready.

In [ ]:
scheduler = AutoCron()

def critical_task():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 🔴 CRITICAL: Security check")

def high_priority_task():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 🟠 HIGH: Data backup")

def normal_task():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 🟢 NORMAL: Log rotation")

def low_priority_task():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 🔵 LOW: Cache cleanup")

# Add tasks with different priorities (higher number = higher priority)
scheduler.add_task(name="critical", func=critical_task, every='10s', priority=10)
scheduler.add_task(name="high", func=high_priority_task, every='10s', priority=7)
scheduler.add_task(name="normal", func=normal_task, every='10s', priority=5)
scheduler.add_task(name="low", func=low_priority_task, every='10s', priority=2)

print("✅ Tasks with different priorities scheduled!")
print("📊 Priority levels: 10 (critical) > 7 (high) > 5 (normal) > 2 (low)")

## 4. Task Dependencies

Run tasks only after other tasks complete.

In [ ]:
scheduler = AutoCron()

def fetch_data():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 1️⃣ Fetching raw data...")
    time.sleep(2)
    return "raw_data.csv"

def process_data():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 2️⃣ Processing data...")
    time.sleep(2)
    return "processed_data.csv"

def generate_report():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 3️⃣ Generating report...")
    time.sleep(1)
    return "report.pdf"

# Add tasks with dependencies
scheduler.add_task(
    name="fetch",
    func=fetch_data,
    every='1h'
)

scheduler.add_task(
    name="process",
    func=process_data,
    every='1h',
    dependencies=["fetch"]  # Runs only after 'fetch' completes
)

scheduler.add_task(
    name="report",
    func=generate_report,
    every='1h',
    dependencies=["process"]  # Runs only after 'process' completes
)

print("✅ Task pipeline with dependencies configured!")
print("📊 Pipeline: fetch → process → report")

## 5. Task Metadata and Tags

Organize and categorize tasks with metadata and tags.

In [ ]:
scheduler = AutoCron()

# Add tasks with metadata and tags
scheduler.add_task(
    name="db_backup",
    func=lambda: print("Backing up database..."),
    every='6h',
    tags=["backup", "database", "critical"],
    metadata={
        "owner": "DevOps Team",
        "database": "production",
        "retention_days": 30,
        "notification_email": "ops@company.com"
    }
)

scheduler.add_task(
    name="log_cleanup",
    func=lambda: print("Cleaning old logs..."),
    every='1d',
    tags=["maintenance", "cleanup"],
    metadata={
        "owner": "Platform Team",
        "cleanup_age_days": 7,
        "disk_threshold_gb": 100
    }
)

scheduler.add_task(
    name="api_monitor",
    func=lambda: print("Monitoring API health..."),
    every='5m',
    tags=["monitoring", "api", "health-check"],
    metadata={
        "owner": "Engineering Team",
        "endpoints": ["api.example.com", "api2.example.com"],
        "alert_threshold_ms": 500
    }
)

print("✅ Tasks with metadata and tags added!\n")

# Display task information
for task in scheduler.tasks:
    print(f"📋 Task: {task.name}")
    print(f"   Tags: {', '.join(task.tags)}")
    print(f"   Owner: {task.metadata.get('owner', 'N/A')}")
    print()

## 6. Max Instances Control

Prevent multiple instances of the same task from running simultaneously.

In [ ]:
scheduler = AutoCron()

def heavy_processing():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Starting heavy processing...")
    time.sleep(10)  # Takes 10 seconds
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Processing complete!")

# Scheduled every 5 seconds but takes 10 seconds to complete
scheduler.add_task(
    name="heavy_task",
    func=heavy_processing,
    every='5s',
    max_instances=1  # Only allow 1 instance to run at a time
)

print("✅ Task configured with max_instances=1")
print("📝 If task is still running when next schedule time arrives, it will skip")

## 7. Task Enable/Disable

Temporarily disable tasks without removing them.

In [ ]:
scheduler = AutoCron()

# Add enabled task
scheduler.add_task(
    name="active_task",
    func=lambda: print("Running..."),
    every='10s',
    enabled=True  # Task is active
)

# Add disabled task
scheduler.add_task(
    name="disabled_task",
    func=lambda: print("This won't run"),
    every='10s',
    enabled=False  # Task is disabled
)

print("✅ Tasks added!\n")

for task in scheduler.tasks:
    status = "🟢 ENABLED" if task.enabled else "🔴 DISABLED"
    print(f"{status}: {task.name}")

## 8. Real-World Example: Complete ETL Pipeline

Combining multiple advanced features in a realistic scenario.

In [ ]:
scheduler = AutoCron()

# Extract: Fetch data from API
def extract_data():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 📥 Extracting data from API...")
    time.sleep(2)
    if random.random() < 0.2:  # 20% failure rate
        raise Exception("API connection failed")
    return "raw_data"

# Transform: Process the data
def transform_data():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] ⚙️  Transforming data...")
    time.sleep(3)
    return "processed_data"

# Load: Save to database
def load_data():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 💾 Loading data to database...")
    time.sleep(2)
    return "success"

# Configure ETL pipeline
scheduler.add_task(
    name="extract",
    func=extract_data,
    every='1h',
    priority=10,              # High priority
    retries=3,                # Retry up to 3 times
    retry_delay=10,           # Wait 10 seconds between retries
    timeout=30,               # 30 second timeout
    tags=["etl", "extract", "api"],
    metadata={
        "owner": "Data Team",
        "api_endpoint": "https://api.example.com/data"
    }
)

scheduler.add_task(
    name="transform",
    func=transform_data,
    every='1h',
    priority=8,
    timeout=60,
    dependencies=["extract"],  # Wait for extract to finish
    tags=["etl", "transform"],
    metadata={
        "owner": "Data Team",
        "transformation_rules": "v2.1"
    }
)

scheduler.add_task(
    name="load",
    func=load_data,
    every='1h',
    priority=7,
    timeout=30,
    dependencies=["transform"],  # Wait for transform to finish
    max_instances=1,            # Only one load at a time
    tags=["etl", "load", "database"],
    metadata={
        "owner": "Data Team",
        "target_database": "analytics_db"
    }
)

print("✅ Complete ETL Pipeline Configured!\n")
print("📊 Pipeline: Extract (retry 3x) → Transform (60s timeout) → Load (1 instance)")
print("🏷️  All tasks tagged: 'etl' for easy filtering")
print("👥 Owner: Data Team")

## Summary

In this demo, you learned:

✅ **Retries** - Automatic retry with configurable delays

✅ **Timeouts** - Prevent tasks from running too long

✅ **Priorities** - Control execution order (1-10 scale)

✅ **Dependencies** - Create task pipelines

✅ **Metadata & Tags** - Organize and categorize tasks

✅ **Max Instances** - Prevent concurrent executions

✅ **Enable/Disable** - Toggle tasks on/off

### Next Steps:
- Check out `03_async_tasks.ipynb` for async/await support
- See `04_persistence.ipynb` for saving and loading tasks
- Explore `05_safe_mode.ipynb` for secure task execution